# High-level Keras (TF) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from common.params import *
from common.utils import *

Using TensorFlow backend.


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.0.6
Numpy:  1.13.1
Tensorflow:  1.2.1
tensorflow
channels_last


In [3]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [4]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [5]:
%%time
# Data into format for library
#x_train, x_test, y_train, y_test = mnist_for_library(channel_first=False, one_hot=True)
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 3.18 s, sys: 1.5 s, total: 4.68 s
Wall time: 30 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 107 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 27.8 ms


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [9]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 40s - loss: 1.8596 - acc: 0.3153    
Epoch 2/10
50000/50000 [==============================] - 39s - loss: 1.4097 - acc: 0.4900    
Epoch 3/10
50000/50000 [==============================] - 41s - loss: 1.2092 - acc: 0.5651    
Epoch 4/10
50000/50000 [==============================] - 41s - loss: 1.0547 - acc: 0.6251    
Epoch 5/10
50000/50000 [==============================] - 40s - loss: 0.9256 - acc: 0.6723    
Epoch 6/10
50000/50000 [==============================] - 40s - loss: 0.8339 - acc: 0.7054    
Epoch 7/10
50000/50000 [==============================] - 40s - loss: 0.7508 - acc: 0.7352    
Epoch 8/10
50000/50000 [==============================] - 41s - loss: 0.6887 - acc: 0.7561    
Epoch 9/10
50000/50000 [==============================] - 40s - loss: 0.6284 - acc: 0.7779    
Epoch 10/10
50000/50000 [==============================] - 41s - loss: 0.5751 - acc: 0.7959    
CPU times: user 3min 34s, sys: 51 s, total: 4min 

In [10]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 1.19 s, sys: 296 ms, total: 1.48 s
Wall time: 2.71 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7738
